# Lab 5: Kaggle Competition

In this lab, I will be building a predictive model for a yelp dataset

## Import Libraries

In [0]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection  import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

In [8]:
import os 
os.getcwd()
os.chdir('C:/Users/adams/Downloads')

FileNotFoundError: ignored

## Import Data 

In [0]:
df_train = pd.read_csv("./Data/yelp_train.csv")
df_test = pd.read_csv("./Data/yelp_test.csv")

## Data Preprocessing

In [0]:
#split into labels and features
X_train = df_train.drop(["review_id", "is_good_rating"], axis=1)
y_train = df_train.is_good_rating

X_test = df_test.drop('review_id', axis=1)

In [0]:
#categorize businesses
# bus_cat_train = X_train['business_categories'].fillna("None")
# bus_cat_train = bus_cat_train.apply(lambda x : x.split(','))

# mlb = MultiLabelBinarizer()
# bus_cat_train = mlb.fit_transform(bus_cat_train)

# #categorize businesses
# bus_cat_test = X_test['business_categories'].fillna("None")
# bus_cat_test = bus_cat_test.apply(lambda x : x.split(','))

# bus_cat_test = mlb.transform(bus_cat_test)

In [0]:
# bus_cat_train #onehotencoded df of types of businesses

In [0]:
# bus_cat_train = pd.DataFrame(bus_cat_train)
# bus_cat_train.head()

In [0]:
X_train.head()

,business_id,user_id,date,cool,funny,useful,text,user_average_stars,user_elite,user_review_count,user_yelping_since,business_categories,business_city,business_latitude,business_longitude,business_review_count,business_average_stars,business_state
0,8zN3nV0zbtE377_XhlvabA,MWahC54Rd_CNv_0VtjCD6A,2017-06-27,0,0,0,"Pool tables, darts, and other neighborhood bar...",2.00,None,5,2013-10-23,"Restaurants, Sports Bars, Bars, American (Trad...",Chandler,33.336050,-111.844343,158,4.0,AZ
1,wJ-PcjemwR44WitIJsDzsA,N-GOto4b3ltfGJ7y-5g-pg,2017-03-25,0,0,0,"This place was alright. However, their classic...",4.43,None,7,2016-07-04,"Beauty & Spas, Nail Salons",Fort Mill,35.058040,-80.937064,26,3.5,SC
2,fP4sMlVw_MvyGNv4xeTGDA,RH3w77UyXaQPiOx2jOE7zg,2017-07-14,0,0,0,First time at Pieology Pizzeria and I was impr...,4.09,None,21,2013-09-05,"Pizza, Restaurants",Las Vegas,36.114884,-115.155604,189,4.0,NV
3,hgWMxKhrnOUd3m5nOUBIkA,g6pwf1E-CylnZQRoaZGGdw,2010-12-13,1,0,0,Its a good facility with lots of equipment and...,3.55,"2013, 2012",83,2010-09-03,"Active Life, Fitness & Instruction, Trainers, ...",Las Vegas,36.025332,-115.120852,316,3.0,NV
4,KMKPusWbBaIORB669W0EeQ,14nl002my6qLaEcqYHLPig,2016-09-12,1,1,1,Very very disappointed! My shawarma chicken wa...,3.75,None,28,2016-03-24,"Mediterranean, Restaurants",Henderson,36.005901,-115.111863,61,3.5,NV


### Differentiate good and bad reviews by keywords

In [0]:
# make some categoricals from text feature
good_tr = X_train.text.apply(lambda x : 1 if re.search(r'good', x) else 0)
great_tr = X_train.text.apply(lambda x : 1 if re.search(r'great', x) else 0)
bad_tr = X_train.text.apply(lambda x : 1 if re.search(r'bad', x) else 0)
amazing_tr = X_train.text.apply(lambda x : 1 if re.search(r'amazing', x) else 0)
awesome_tr = X_train.text.apply(lambda x : 1 if re.search(r'awesome', x) else 0)
best_tr= X_train.text.apply(lambda x : 1 if re.search(r'best', x) else 0)
worst_tr= X_train.text.apply(lambda x : 1 if re.search(r'worst', x) else 0)
love_tr= X_train.text.apply(lambda x : 1 if re.search(r'love', x) else 0)
hate_tr= X_train.text.apply(lambda x : 1 if re.search(r'hate', x) else 0)
favorite_tr= X_train.text.apply(lambda x : 1 if re.search(r'favorite', x) else 0)
disgusting_tr=X_train.text.apply(lambda x : 1 if re.search(r'disgusting', x) else 0)
awful_tr=X_train.text.apply(lambda x : 1 if re.search(r'awful', x) else 0)


In [0]:
# turn into dataframe
common_words_train = pd.DataFrame({'good' : good_tr, 'bad' : bad_tr, 'great' : great_tr, 'amazing' : amazing_tr, 
                                   'awesome': awesome_tr, 'best':best_tr, 'worst':worst_tr, 'love':love_tr, 'hate':hate_tr,
                                  'favorite':favorite_tr, 'disgusting':disgusting_tr, 'awful':awful_tr} )

In [0]:
# make some categoricals from text feature repeat for test data
good_tst = X_test.text.apply(lambda x : 1 if re.search(r'good', x) else 0)
great_tst = X_test.text.apply(lambda x : 1 if re.search(r'great', x) else 0)
bad_tst = X_test.text.apply(lambda x : 1 if re.search(r'bad', x) else 0)
amazing_tst = X_test.text.apply(lambda x : 1 if re.search(r'amazing', x) else 0)
awesome_tst = X_test.text.apply(lambda x : 1 if re.search(r'awesome', x) else 0)
best_tst= X_test.text.apply(lambda x : 1 if re.search(r'best', x) else 0)
worst_tst= X_test.text.apply(lambda x : 1 if re.search(r'worst', x) else 0)
love_tst= X_test.text.apply(lambda x : 1 if re.search(r'love', x) else 0)
hate_tst= X_test.text.apply(lambda x : 1 if re.search(r'hate', x) else 0)
favorite_tst= X_test.text.apply(lambda x : 1 if re.search(r'favorite', x) else 0)
disgusting_tst=X_test.text.apply(lambda x : 1 if re.search(r'disgusting', x) else 0)
awful_tst=X_test.text.apply(lambda x : 1 if re.search(r'awful', x) else 0)


In [0]:
# turn into dataframe
common_words_tst = pd.DataFrame({'good' : good_tst, 'bad' : bad_tst, 'great' : great_tst, 'amazing' : amazing_tst, 
                                 'awesome': awesome_tst, 'best':best_tst, 'worst':worst_tst, 'love':love_tst,
                                'hate':hate_tst, 'favorite':favorite_tst, 'disgusting':disgusting_tst, 'awful':awful_tst})

### Differentiate elite yelpers from non-elite

In [0]:
#turn user elite column into number of times user has been user elite
user_elite_tr = X_train.user_elite.apply(lambda x : 0 if x == 'None' else len(x.split(",")))
user_elite_tst = X_test.user_elite.apply(lambda x : 0 if x == 'None' else len(x.split(",")))

### Calculate diff from start of yelp to the year of the review

In [0]:
review_date_tr = X_train.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)
start_date_tr = X_train.user_yelping_since.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)

date_diff_tr = review_date_tr-start_date_tr

In [0]:
review_date_tst = X_test.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)
start_date_tst = X_test.user_yelping_since.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)

date_diff_tst = review_date_tst-start_date_tst

## Update Training Data and Test Data

In [0]:
new_X_train = X_train.drop(['text', 'business_id', 'user_id', 'business_categories', 'business_latitude', 
              'business_longitude', 'business_state', 'business_city', 'date', 'user_yelping_since'], axis=1)
new_X_train['user_elite'] = user_elite_tr
new_X_train['date_diff'] = date_diff_tr
new_X_train = new_X_train.join(common_words_train)
new_X_train.head()

,cool,funny,useful,user_average_stars,user_elite,user_review_count,business_review_count,business_average_stars,date_diff,amazing,awesome,bad,best,favorite,good,great,hate,love,worst
0,0,0,0,2.00,0,5,158,4.0,4,0,0,0,0,1,0,0,0,0,0
1,0,0,0,4.43,0,7,26,3.5,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,4.09,0,21,189,4.0,4,0,0,0,0,0,0,0,0,0,0
3,1,0,0,3.55,2,83,316,3.0,0,0,0,0,0,0,1,1,0,1,0
4,1,1,1,3.75,0,28,61,3.5,0,0,0,0,0,0,0,0,0,0,0


In [0]:
new_X_test = X_test.drop(['text', 'business_id', 'user_id', 'business_categories', 'business_latitude', 
              'business_longitude', 'business_state', 'business_city', 'date', 'user_yelping_since'], axis=1)
new_X_test['user_elite'] = user_elite_tst
new_X_test['date_diff'] = date_diff_tst
new_X_test = new_X_test.join(common_words_tst)
new_X_test.head()

,cool,funny,useful,user_average_stars,user_elite,user_review_count,business_review_count,business_average_stars,date_diff,amazing,awesome,bad,best,favorite,good,great,hate,love,worst
0,2,1,2,3.80,4,369,81,4.0,4,0,0,1,0,0,0,0,0,0,0
1,1,0,2,3.80,10,483,18,5.0,4,0,1,0,0,0,0,0,0,0,0
2,1,1,1,4.14,0,7,572,4.0,1,1,0,0,1,0,0,0,0,0,0
3,1,1,2,4.26,2,131,33,4.0,1,0,0,0,0,0,0,0,0,0,0
4,7,6,11,3.83,6,212,558,4.0,3,1,0,0,1,0,1,1,0,1,0



## Train ML Models

In [0]:
#Can't get to work right now, very very slow, unsure why.
from sklearn.neighbors import KNeighborsClassifier
def kn_model(X, y):
    neighbors = list(range(1,20))
    distances = [1, 2]
    param_grid = {"n_neighbors":neighbors, "p":distances}
    scores=['accuracy']
    for score in scores:
        clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring=score)
        grid_result=clf.fit(X, y)
        best_params = grid_result.best_params_
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()
    return grid_result


In [0]:
kn_grid=kn_model(new_X_train, y_train )

In [0]:
new_X_train.head()

### Logistic Regression 

In [0]:
lr = LogisticRegression(solver='liblinear')
cv_results = cross_validate(lr, new_X_train, y_train, cv=5)
cv_results['test_score']

array([0.81056645, 0.81058728, 0.81147917, 0.80743349, 0.81035022])

In [0]:
lr.fit(new_X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
y_pred = lr.predict(new_X_train)
accuracy_score(y_pred, y_train)

0.8101833333333334

In [0]:
submission = pd.DataFrame(index=df_test.review_id)
submission['is_good_rating'] = lr.predict(new_X_test)

In [0]:
submission.reset_index().to_csv('submission.csv', index=False)

### Linear SVC 

In [0]:
svm = LinearSVC()
cv_results = cross_validate(svm, new_X_train, y_train, cv=5)
cv_results['test_score']

C:\Users\Adam\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


KeyboardInterrupt: 

In [0]:
svm = LinearSVC()
svm.fit(new_X_train, y_train)

In [0]:
accuracy_score(svm.predict(new_X_train), y_train)

0.7427916666666666

### Decision Tree

In [0]:
clf = DecisionTreeClassifier()
cv_results = cross_validate(clf, new_X_train, y_train, cv=5)
cv_results['test_score']

array([0.73083894, 0.7313806 , 0.729375  , 0.72880685, 0.72647347])

In [0]:
clf = DecisionTreeClassifier()
clf.fit(new_X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
accuracy_score(clf.predict(new_X_train), y_train)

0.9994166666666666